<a href="https://colab.research.google.com/github/alnabenny/fake-news-detection/blob/main/notebooks/fake_news_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install nlpaug

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.0 MB/s eta 0:00:00


In [3]:
import nltk

# Download all necessary resources
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')

# Manually install the missing package
nltk.download('averaged_perceptron_tagger_eng')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [4]:
import nlpaug.augmenter.word as naw

# Create an augmenter that replaces words with synonyms
aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.2)

# Example news headline
text = "NASA verifies human settlement on Mars by next year."

# Generate an augmented version
augmented_text = aug.augment(text)

print("🔹 Original:", text)
print("🔹 Augmented:", augmented_text)


🔹 Original: NASA verifies human settlement on Mars by next year.
🔹 Augmented: ['NASA verifies human settlement on Mars by adjacent twelvemonth.']


In [5]:
!ls /content/drive/MyDrive/


'03-Nov-2020 10:36:47 AM.jpeg'
'03-Nov-2020 10:56:19 AM.jpeg'
'03-Nov-2020 11:06:56 AM.jpeg'
'03-Nov-2020 11:07:50 AM.jpeg'
'03-Nov-2020 11:39:55 AM.jpeg'
'03-Nov-2020 11:42:17 AM.jpeg'
'03-Nov-2020 12:15:46 PM.jpeg'
'03-Nov-2020 3:58:46 PM.jpeg'
'04-Nov-2020 12:17:56 PM.jpeg'
'05-Nov-2020 1:05:46 PM.jpeg'
'07-Jan-2021 11:19:29 AM.jpeg'
'07-Jan-2021 11:20:48 AM.jpeg'
'07-Jan-2021 11:24:23 AM.jpeg'
'07-Jan-2021 11:36:42 AM.jpeg'
'07-Jan-2021 11:37:49 AM.jpeg'
'10 alna.pdf'
'10-Nov-2020 12:13:23 PM.jpeg'
'10-Nov-2020 3:11:40 PM (1).jpeg'
'10-Nov-2020 3:11:40 PM (2).jpeg'
'10-Nov-2020 3:11:40 PM.jpeg'
'10-Nov-2020 7:59:21 PM.jpeg'
'10-Nov-2020 8:00:49 PM.jpeg'
'10th alna.pdf'
'12 alna.pdf'
'12th alna.pdf'
 12th.pdf
'1438938659STD V.pdf'
 1600437185915..jpg
 1600437267230..jpg
 16149387476476610779400138937177.jpg
 20210201_203546_copy_150x200.jpg
 20220729_223131.jpg
'21-Oct-2020 11:42:12 AM.jpeg'
'21-Oct-2020 2:44:54 PM.jpeg'
'21-Oct-2020 2:45:19 PM.jpeg'
'21-Oct-2020 2:47:44 PM (1).jpeg

In [6]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Fake.csv')

# Print column names
print("📝 Column Names:", df.columns)


📝 Column Names: Index(['title', 'text', 'subject', 'date'], dtype='object')


In [7]:
import pandas as pd

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Fake.csv')

# Add a 'label' column (0 for fake news)
df['label'] = 0

# Save the updated dataset
df.to_csv('/content/drive/MyDrive/Fake_labeled.csv', index=False)

print("✅ 'label' column added successfully! New file saved as 'Fake_labeled.csv'")


✅ 'label' column added successfully! New file saved as 'Fake_labeled.csv'


In [8]:
import pandas as pd
import nlpaug.augmenter.word as naw

# Load the updated dataset
df = pd.read_csv('/content/drive/MyDrive/Fake_labeled.csv')

# Create a synonym augmenter
aug = naw.SynonymAug(aug_src='wordnet', aug_p=0.2)

# Apply augmentation to a subset (only augment a few samples to avoid overloading)
augmented_texts = []
augmented_labels = []

for i in range(len(df) // 5):  # Augment only 20% of the data
    original_text = df.iloc[i]["text"]
    augmented_text = aug.augment(original_text)

    augmented_texts.append(augmented_text)
    augmented_labels.append(df.iloc[i]["label"])  # Keep the same label

# Convert to DataFrame and add to original data
aug_df = pd.DataFrame({"text": augmented_texts, "label": augmented_labels})
df = pd.concat([df, aug_df], ignore_index=True)

# Save the new dataset
df.to_csv('/content/drive/MyDrive/Fake_augmented.csv', index=False)

print("✅ Data Augmentation Completed. New dataset saved as 'Fake_augmented.csv'")


✅ Data Augmentation Completed. New dataset saved as 'Fake_augmented.csv'


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the augmented dataset
df = pd.read_csv('/content/drive/MyDrive/Fake_augmented.csv')

# Check for missing values and drop them if any
df = df.dropna()

# Split dataset into train & test sets (80% train, 20% test)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

print(f"✅ Dataset Split: {len(train_texts)} training samples, {len(test_texts)} testing samples")


✅ Dataset Split: 18784 training samples, 4697 testing samples


In [10]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

print(f"✅ Labels encoded: {set(train_labels)}")


✅ Labels encoded: {np.int64(0)}


In [11]:
print(df['label'].value_counts())

label
0    23481
Name: count, dtype: int64


In [12]:
df_original = pd.read_csv("/content/drive/MyDrive/Fake.csv")
print("Fake.csv samples:", len(df_original))

df_real = pd.read_csv("/content/drive/MyDrive/True.csv")
print("True.csv samples:", len(df_real))


Fake.csv samples: 23481
True.csv samples: 21417


In [13]:
import pandas as pd

# Load original datasets
df_fake = pd.read_csv("/content/drive/MyDrive/Fake.csv")
df_real = pd.read_csv("/content/drive/MyDrive/True.csv")

# Assign labels
df_fake["label"] = 0
df_real["label"] = 1

# Combine and shuffle
df_combined = pd.concat([df_fake, df_real], ignore_index=True)
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Keep only the required columns
df_combined = df_combined[["text", "label"]]

# Save for reference
df_combined.to_csv("/content/drive/MyDrive/combined_dataset.csv", index=False)

print("✅ Combined dataset created with shape:", df_combined.shape)
print(df_combined["label"].value_counts())


✅ Combined dataset created with shape: (44898, 2)
label
0    23481
1    21417
Name: count, dtype: int64


In [14]:
from sklearn.model_selection import train_test_split

# Reload the balanced dataset
df = pd.read_csv('/content/drive/MyDrive/combined_dataset.csv')

# Check and drop any missing values
df = df.dropna()

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

print(f"✅ Split done: {len(train_texts)} train samples, {len(test_texts)} test samples")

✅ Split done: 35918 train samples, 8980 test samples


In [15]:
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer

# Encode labels (0 and 1 are fine, but this ensures consistency)
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

# Load DeBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

print("✅ Labels encoded and tokenizer loaded.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


✅ Labels encoded and tokenizer loaded.


In [16]:
# Tokenize the training and test texts
train_encodings = tokenizer(
    list(train_texts),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

test_encodings = tokenizer(
    list(test_texts),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

print("✅ Texts tokenized.")

✅ Texts tokenized.


In [17]:
import torch
from torch.utils.data import Dataset

# Create a PyTorch dataset
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Wrap train and test datasets
train_dataset = FakeNewsDataset(train_encodings, train_labels)
test_dataset = FakeNewsDataset(test_encodings, test_labels)

print("✅ PyTorch Datasets ready.")


✅ PyTorch Datasets ready.


In [18]:
from torch.utils.data import DataLoader

# Set batch size
batch_size = 2  # Small to avoid memory issues in free Colab

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

print("✅ DataLoaders created.")

✅ DataLoaders created.


In [19]:
from transformers import AutoModelForSequenceClassification

# Load DeBERTa-v3-base model with 2 output labels
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    num_labels=2
)

# Move model to device (GPU if available)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(f"✅ Model loaded and moved to device: {device}")

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Model loaded and moved to device: cpu


In [20]:
import torch.optim as optim
import torch.nn as nn

# Optimizer (use AdamW from torch)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

# Loss function
loss_fn = nn.CrossEntropyLoss()

print("✅ Optimizer and Loss Function set up.")

✅ Optimizer and Loss Function set up.


In [21]:
def train_model(model, train_dataloader, optimizer, loss_fn, device, epochs=2):
    model.train()
    history = {'loss': [], 'accuracy': []}

    for epoch in range(epochs):
        total_loss = 0
        correct = 0
        total = 0

        print(f"\n🚀 Epoch {epoch + 1}/{epochs} Training...")

        for batch in train_dataloader:
            # Move inputs to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Compute loss
            loss = loss_fn(logits, labels)
            total_loss += loss.item()

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accuracy
            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        avg_loss = total_loss / len(train_dataloader)
        accuracy = correct / total * 100

        history['loss'].append(avg_loss)
        history['accuracy'].append(accuracy)

        print(f"📊 Epoch {epoch + 1}: Loss = {avg_loss:.4f}, Accuracy = {accuracy:.2f}%")

    return history


In [22]:
import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
history = train_model(model, train_dataloader, optimizer, loss_fn, device, epochs=1)




🚀 Epoch 1/1 Training...


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history['accuracy'], marker='o')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history['loss'], marker='o', color='orange')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()
